In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from numpy import array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Embedding
from tensorflow.keras.utils import to_categorical

#케라스의 텍스트 전처리와 관련한 함수중 text_to_word_sequence 함수를 불러 옵니다.
from tensorflow.keras.preprocessing.text import text_to_word_sequence
 
# 전처리할 텍스트를 정합니다.
train = pd.read_csv('train_mydataset_6000.csv')
test = pd.read_csv('test_mydataset_1500.csv')

train_text = np.array(train)
x_train = [] #학습셋 제목
y_train = [] #학습셋 라벨
for i in range(len(train_text)):
    x_train.append(train_text[i][1])
    y_train.append(train_text[i][2])

test_text = np.array(test)
x_test = [] #테스트셋 제목
y_test = [] #테스트셋 라벨
for i in range(len(test_text)):
    x_test.append(test_text[i][1])
    y_test.append(test_text[i][2])
    
token = Tokenizer()
token.fit_on_texts(x_train + x_test)
token.fit_on_texts(x_test)
token.fit_on_texts(x_train)
x_token = token.texts_to_sequences(x_train + x_test) #토큰화

train_token = token.texts_to_sequences(x_train)
test_token = token.texts_to_sequences(x_test)

padded_x_train = pad_sequences(train_token, 14)
padded_x_test = pad_sequences(test_token, 14)
padded_x = pad_sequences(x_token, 14)

#원핫인코딩
y = y_train + y_test
y_encoded = tf.keras.utils.to_categorical(y)
classes = np.array(y)
y_train = np.array(y_train)
y_test = np.array(y_test)
y = np.array(y)

#임베딩에 입력될 단어의 수를 지정합니다.
word_size = len(token.word_index) +1
 
#단어 임베딩을 포함하여 딥러닝 모델을 만들고 결과를 출력합니다.
model = Sequential()
model.add(Embedding(word_size, 8, input_length=14))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(padded_x, y, epochs=20) #, validation_data=(padded_x_test, y_test)

print("\n Accuracy: %.4f\n" % (model.evaluate(padded_x_test, y_test)[1]))

#출력
print("첫번째 입력: " , x_train[0])
print("첫번째 one-hot 출력: ", y_encoded[0])
print("첫번째 토큰 결과: ", x_token[0])
trainMax = max(len(i) for i in train_token)
print("학습셋 제목 최대 길이: ", trainMax)
testMax = max(len(i) for i in test_token)
print("테스트셋 제목 최대 길이: ", testMax)
print("첫번째 패딩 토큰: ", padded_x[0])
print("Word Size: ", word_size)
